# webdriver 경로 바꿔주기

In [13]:
dpath='C:/asi/project/chromedriver_win32 (3)/chromedriver'

In [14]:
from bs4 import BeautifulSoup as bsp
from selenium import webdriver
driver=webdriver.Chrome(dpath)

In [15]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup as bsp
import matplotlib
import matplotlib.pyplot as plt
import sqlite3
import re
from datetime import datetime, timedelta,date
import sys
from dateutil.parser import parse

In [16]:
def getData(whr=''):
    dbname='basenote.db'
    with sqlite3.connect(dbname) as conn:
        cursor=conn.cursor()
        sql='select * from main '+whr
        res=cursor.execute(sql).fetchall()
    return(res)

In [17]:
def insInfo(d):
    dbname='basenote.db'
    with sqlite3.connect(dbname) as conn:
        cursor=conn.cursor()
        sql='select * from info where id="'+d[0]+'"'
        res=cursor.execute(sql).fetchall()
        if(len(res)>0):
            print("이미 존재 합니다.:"+d[1])
        else:
            sql='insert into info (id,product,year,gender,availability,rating,house,topnotes,heartnotes,basenotes,uninotes) values(?,?,?,?,?,?,?,?,?,?,?)'
            res=cursor.execute(sql,d)
            print('.',end='')

In [18]:
def insReview(d):
    dbname='basenote.db'
    with sqlite3.connect(dbname) as conn:
        cursor=conn.cursor()
        sql='select * from review where reviewid="'+d[1]+'"'
        res=cursor.execute(sql).fetchall()
        if(len(res)>0):
            print("이미 존재 합니다.:"+d[1])
        else:
            sql='insert into review (productid,reviewid,author,date,thumbrating,content) values(?,?,?,?,?,?)'
            res=cursor.execute(sql,d)
            print('.',end='')

In [19]:
def classifynote(notes):
    tnotes=[]
    hnotes=[]
    bnotes=[]
    uninotes=[]
    if 'Top Notes' and 'Heart Notes' and 'Base notes' in notes:
        tidx=notes.index('Top Notes')
        hidx=notes.index('Heart Notes')
        bidx=notes.index('Base notes')
        
        a=hidx-tidx
        b=bidx-hidx
        
        for i in range(1,a):
            tnotes.append(notes[tidx+i])
        for i in range(1,b):
            hnotes.append(notes[hidx+i])
        for i in range(1,len(notes)-bidx):
            bnotes.append(notes[bidx+i])
    else:
        
        for i in notes:
            uninotes.append(i)
    if len(tnotes)==0:
        tnotes.append('-')
    if len(hnotes)==0:
        hnotes.append('-')
    if len(bnotes)==0:
        bnotes.append('-')
    if len(uninotes)==0:
        uninotes.append('-')
    return tnotes,hnotes,bnotes,uninotes

In [20]:
def getReview(id):
    url='https://www.basenotes.net/fragrancereviews/fragrance/{}'.format(id)
    driver.get(url)
    driver.implicitly_wait(3)
    html=driver.page_source
    src=bsp(html,'html.parser')
    cts=src.select('div.dirleft-content div.reviewmain')
    reviews=[]
    for ct in cts:
        review=[]
        reviewid=ct['id'].replace('review-','')
        author=ct.select('div.reviewauthor b')[0].text
        date_=ct.select('div.reviewdate')[0].text
        if ('last edited: '  in date_):
            dateidx = re.search('last edited: ', date_).span()[-1]
            date_ =  date_[dateidx:-2]    
        print(date_)
        dt = parse(date_)
        date_1=dt.date()
        date=date_1.strftime('%Y-%m-%d') 
        content=ct.select('div.reviewblurb')[0].text
        rating_=ct.select('div.reviewauthor img')[1]['src']
        rating=rating_.replace('https://www.basenotes.net/assets/icons/','').replace('.png','').replace('review','')
        review.append(reviewid)
        review.append(author)
        review.append(date)
        review.append(rating)
        review.append(content)
        reviews.append(review)
    return reviews

In [21]:
def scrapMain(url):
    #향수 main리스트(이름,url,ID)
    driver.get(url)
    #driver.implicitly_wait(3)
    html=driver.page_source
    src=bsp(html,'html.parser')
    tgts=src.select('tbody tr td h3 a')
    for tgt in tgts:
        title=tgt.text
        item_url=tgt['href']
        key=makeKey(item_url)
        #driver.get(item_url)
        #driver.implicitly_wait(3)
        #html=driver.page_source
        #src=bsp(html,'html.parser')
        dm=(key,title,item_url)
        insMain(dm)
        print(dm)

In [22]:
def scrapReview():
    #리뷰   
    allurl=getData()
    for urls in allurl:
        key=urls[1]
        reviews=getReview(key)
        for review in reviews:
            #key=review[0]
            reviewid=review[0]
            author=review[1]
            date=review[2]
            thumbrating=review[3]
            content=review[4]
            dr=(key,reviewid,author,date,thumbrating,content)
            insReview(dr)
            print('-----reviewid: {}완료-----'.format(reviewid))

## 자기 부분 주석 해제 후 실행

In [23]:
def scrapInfo():
    
#Item 세부 정보 (연도,젠더,이용가능성,하우스,레이팅)
    
    allurl=getData()
    
#     for urls in allurl[15000:24000]: # 호영
    for urls in allurl[24000:27000]: # 다영
#     for urls in allurl[27000:42000]: # 송재
#     for urls in allurl[42000:]: # 기범
        key=urls[1]
        title=urls[2]
        item_url=urls[3]
        driver.get(item_url)
        driver.implicitly_wait(3)
        html=driver.page_source
        src=bsp(html,'html.parser')
        
        if src.select('div.peoplelist tr'): 
            trs=src.select('div.peoplelist tr')
            infos=[]
            for tr in trs:
                infos.append(tr.text)
                #print(tr.text)

            if 'Year of Launch' in infos[0]:
                year=infos[0].replace('Year of Launch','')
                gender=infos[1].replace('GenderShared / ','')
                availability=infos[2].replace('Availability','')
                house=infos[5].replace('House','')
                if trs[3].select('meta'):
                    rating=trs[3].select('meta')[0]['content']
                else:
                    rating=trs[3].select('small')[0].text
            else:
                year='-'
                gender=infos[0].replace('GenderShared / ','')
                availability=infos[1].replace('Availability','')
                house=infos[4].replace('House','')
                if trs[2].select('meta'):
                    rating=trs[2].select('meta')[0]['content']
                else:
                    rating=trs[2].select('small')[0].text

            #노트정보(top,heart,base별로 정리/구분안된거는 uninote로)
            if src.select('div.dirright50 li'): 
                notes=src.select('div.dirright50 li')
                tops=[]
                hearts=[]
                bases=[]
                unis=[]
                notes_=[]
                for note in notes:
                    notes_.append(note.text)
                a=','.join(notes_)
                a_=a.replace('                                        ','')
                a__=a_.replace('\n\n\n','')
                b=a__.replace('\n\n','')
                c=b.replace('\n',',')
                d=c.replace(', ',',')
                e=d.replace('Top Notes','Top Notes,').replace(',,',',').replace(' ,',',')
                fnotes=e.split(',')
                tops,hearts,bases,unis=classifynote(fnotes)
                tops=','.join(tops)
                hearts=','.join(hearts)
                bases=','.join(bases)
                unis=','.join(unis)
                dinfo=(key,title,year,gender,availability,rating,house,tops,hearts,bases,unis)
                insInfo(dinfo)
                print('-----{} key완료-----'.format(key))
            else:
                tops='-'
                hearts='-'
                bases='-'
                unis='-'
                dinfo=(key,title,year,gender,availability,rating,house,tops,hearts,bases,unis)
                insInfo(dinfo)
                print('-----{} key완료-----'.format(key))

        else: 
            year='-'
            gender='-'
            availability='-'
            rating='-'
            house='-'
            tops='-'
            hearts='-'
            bases='-'
            unis='-'
            dinfo=(key,title,year,gender,availability,rating,house,tops,hearts,bases,unis)
            insInfo(dinfo)
            print('-----{} key완료-----'.format(key))
           # pass

In [24]:
scrapInfo()

.-----26132594 key완료-----
.-----26136513 key완료-----
.-----26147751 key완료-----
.-----26154644 key완료-----
.-----26153512 key완료-----
.-----26158959 key완료-----
.-----26155677 key완료-----
.-----26159827 key완료-----
.-----10210201 key완료-----
.-----10210393 key완료-----
.-----26124806 key완료-----
.-----10211173 key완료-----
.-----10211246 key완료-----
.-----26125246 key완료-----
.-----26160544 key완료-----
.-----26140196 key완료-----
.-----26160167 key완료-----
.-----26143371 key완료-----
.-----26130172 key완료-----
.-----26143882 key완료-----
.-----26125525 key완료-----
.-----26129563 key완료-----
.-----26147312 key완료-----
.-----26148506 key완료-----
.-----26140355 key완료-----
.-----26150135 key완료-----
.-----26125892 key완료-----
.-----26143778 key완료-----
.-----26134769 key완료-----
.-----26134770 key완료-----
.-----26160811 key완료-----
.-----26156098 key완료-----
.-----26134028 key완료-----
.-----26135408 key완료-----
.-----26134346 key완료-----
.-----26137327 key완료-----
.-----26146864 key완료-----
.-----10213335 key완료-----
.-----261543

.-----26130524 key완료-----
.-----26139388 key완료-----
.-----26145398 key완료-----
.-----26145399 key완료-----
.-----26144295 key완료-----
.-----26155361 key완료-----
.-----26154051 key완료-----
.-----26149264 key완료-----
.-----26124401 key완료-----
.-----26140377 key완료-----
.-----10212163 key완료-----
.-----26142901 key완료-----
.-----10212167 key완료-----
.-----26145951 key완료-----
.-----26163057 key완료-----
.-----26128753 key완료-----
.-----26142385 key완료-----
.-----26121007 key완료-----
.-----26138896 key완료-----
.-----26138899 key완료-----
.-----26121008 key완료-----
.-----26134850 key완료-----
.-----26134035 key완료-----
.-----26134034 key완료-----
.-----26142132 key완료-----
.-----26142131 key완료-----
.-----26149189 key완료-----
.-----26145741 key완료-----
.-----26152344 key완료-----
.-----26152429 key완료-----
.-----26121009 key완료-----
.-----26141248 key완료-----
.-----26149515 key완료-----
.-----26144676 key완료-----
.-----26140426 key완료-----
.-----26146519 key완료-----
.-----26122969 key완료-----
.-----26138895 key완료-----
.-----261366

.-----26134510 key완료-----
.-----26151920 key완료-----
.-----26144299 key완료-----
.-----26135317 key완료-----
.-----26151598 key완료-----
.-----26155696 key완료-----
.-----26143125 key완료-----
.-----26148808 key완료-----
.-----26138377 key완료-----
.-----26138376 key완료-----
.-----26134117 key완료-----
.-----26134120 key완료-----
.-----26134121 key완료-----
.-----26132561 key완료-----
.-----26138849 key완료-----
.-----26149049 key완료-----
.-----26139316 key완료-----
.-----26135552 key완료-----
.-----26135182 key완료-----
.-----26158751 key완료-----
.-----26139317 key완료-----
.-----26139321 key완료-----
.-----26141047 key완료-----
.-----26146331 key완료-----
.-----26133785 key완료-----
.-----26133893 key완료-----
.-----26133894 key완료-----
.-----26133892 key완료-----
.-----26133888 key완료-----
.-----26123284 key완료-----
.-----26140274 key완료-----
.-----26129166 key완료-----
.-----26138061 key완료-----
.-----26145905 key완료-----
.-----26123555 key완료-----
.-----26128267 key완료-----
.-----26162611 key완료-----
.-----26124408 key완료-----
.-----261552

.-----26153799 key완료-----
.-----26147788 key완료-----
.-----26144168 key완료-----
.-----26148899 key완료-----
.-----26149557 key완료-----
.-----26149584 key완료-----
.-----26156883 key완료-----
.-----26162313 key완료-----
.-----10212482 key완료-----
.-----26151196 key완료-----
.-----26135247 key완료-----
.-----10212746 key완료-----
.-----26154424 key완료-----
.-----26136413 key완료-----
.-----26154926 key완료-----
.-----26151965 key완료-----
.-----26148856 key완료-----
.-----26140100 key완료-----
.-----26145975 key완료-----
.-----26143152 key완료-----
.-----26156661 key완료-----
.-----26156334 key완료-----
.-----26133582 key완료-----
.-----26158840 key완료-----
.-----10212784 key완료-----
.-----26146972 key완료-----
.-----26146692 key완료-----
.-----26124552 key완료-----
.-----26145805 key완료-----
.-----26146060 key완료-----
.-----26158309 key완료-----
.-----26160339 key완료-----
.-----26132160 key완료-----
.-----26132159 key완료-----
.-----26127772 key완료-----
.-----26144797 key완료-----
.-----26150664 key완료-----
.-----26140269 key완료-----
.-----261496

.-----26123506 key완료-----
.-----26129183 key완료-----
.-----26129181 key완료-----
.-----26129182 key완료-----
.-----26149134 key완료-----
.-----26149133 key완료-----
.-----26133811 key완료-----
.-----26158182 key완료-----
.-----26132180 key완료-----
.-----26151832 key완료-----
.-----26133630 key완료-----
.-----26136707 key완료-----
.-----10212555 key완료-----
.-----26138654 key완료-----
.-----26158398 key완료-----
.-----26151586 key완료-----
.-----26155468 key완료-----
.-----26134227 key완료-----
.-----26130401 key완료-----
.-----26139193 key완료-----
.-----26149652 key완료-----
.-----26127400 key완료-----
.-----10212508 key완료-----
.-----26158741 key완료-----
.-----26121041 key완료-----
.-----26132766 key완료-----
.-----26153517 key완료-----
.-----26148509 key완료-----
.-----26153052 key완료-----
.-----26153051 key완료-----
.-----26153516 key완료-----
.-----26122728 key완료-----
.-----26132191 key완료-----
.-----10211390 key완료-----
.-----10213235 key완료-----
.-----26141596 key완료-----
.-----26134556 key완료-----
.-----26140131 key완료-----
.-----102107

.-----26157772 key완료-----
.-----26152744 key완료-----
.-----26138335 key완료-----
.-----26127786 key완료-----
.-----26159756 key완료-----
.-----26133651 key완료-----
.-----10211381 key완료-----
.-----26126811 key완료-----
.-----26140076 key완료-----
.-----26157494 key완료-----
.-----26141978 key완료-----
.-----26143706 key완료-----
.-----26160860 key완료-----
.-----26156476 key완료-----
.-----26157081 key완료-----
.-----26127401 key완료-----
.-----26131198 key완료-----
.-----26161444 key완료-----
.-----26127787 key완료-----
.-----26129231 key완료-----
.-----26124540 key완료-----
.-----26121045 key완료-----
.-----26121046 key완료-----
.-----10212168 key완료-----
.-----26143188 key완료-----
.-----26143189 key완료-----
.-----26149963 key완료-----
.-----26121047 key완료-----
.-----26121048 key완료-----
.-----26121049 key완료-----
.-----26146886 key완료-----
.-----26128286 key완료-----
.-----26147277 key완료-----
.-----26145324 key완료-----
.-----26139008 key완료-----
.-----26145013 key완료-----
.-----26129591 key완료-----
.-----26149675 key완료-----
.-----261543

.-----26133145 key완료-----
.-----26149039 key완료-----
.-----26123650 key완료-----
.-----26132414 key완료-----
.-----26121044 key완료-----
.-----26125623 key완료-----
.-----26145681 key완료-----
.-----26157437 key완료-----
.-----26160188 key완료-----
.-----26141015 key완료-----
.-----26142510 key완료-----
.-----26145543 key완료-----
.-----10212197 key완료-----
.-----26142485 key완료-----
.-----26156680 key완료-----
.-----26150426 key완료-----
.-----26153594 key완료-----
.-----10213107 key완료-----
.-----26147469 key완료-----
.-----26122798 key완료-----
.-----26121064 key완료-----
.-----26127796 key완료-----
.-----26135272 key완료-----
.-----26155661 key완료-----
.-----26126926 key완료-----
.-----26127404 key완료-----
.-----26128429 key완료-----
.-----26143224 key완료-----
.-----26140073 key완료-----
.-----26121065 key완료-----
.-----26128529 key완료-----
.-----26141580 key완료-----
.-----26137091 key완료-----
.-----26129640 key완료-----
.-----26138962 key완료-----
.-----26126228 key완료-----
.-----26131617 key완료-----
.-----26137860 key완료-----
.-----261450

.-----26125198 key완료-----
.-----26127802 key완료-----
.-----26137190 key완료-----
.-----26148746 key완료-----
.-----26127803 key완료-----
.-----26127233 key완료-----
.-----26130108 key완료-----
.-----26146684 key완료-----
.-----26146607 key완료-----
.-----26160044 key완료-----
.-----26156002 key완료-----
.-----26150427 key완료-----
.-----26126398 key완료-----
.-----26148438 key완료-----
.-----26143550 key완료-----
.-----26136551 key완료-----
.-----26147716 key완료-----
.-----26124546 key완료-----
.-----10212279 key완료-----
.-----26149298 key완료-----
.-----26162328 key완료-----
.-----26126298 key완료-----
.-----26139608 key완료-----
.-----10212996 key완료-----
.-----26131656 key완료-----
.-----26134867 key완료-----
.-----26131844 key완료-----
.-----26129695 key완료-----
.-----26144668 key완료-----
.-----26121077 key완료-----
.-----26131870 key완료-----
.-----26160992 key완료-----
.-----26127804 key완료-----
.-----26156346 key완료-----
.-----26155976 key완료-----
.-----26158047 key완료-----
.-----26161137 key완료-----
.-----26147768 key완료-----
.-----261618

.-----26149758 key완료-----
.-----26144287 key완료-----
.-----26142052 key완료-----
.-----26150820 key완료-----
.-----26126883 key완료-----
.-----26140440 key완료-----
.-----26128803 key완료-----
.-----26128547 key완료-----
.-----26130629 key완료-----
.-----26162721 key완료-----
.-----26136113 key완료-----
.-----26131582 key완료-----
.-----26150557 key완료-----
.-----26149826 key완료-----
.-----26149396 key완료-----
.-----26145244 key완료-----
.-----26161807 key완료-----
.-----26121098 key완료-----
.-----26150742 key완료-----
.-----26129381 key완료-----
.-----26150252 key완료-----
.-----26161878 key완료-----
.-----26127807 key완료-----
.-----26127316 key완료-----
.-----26131035 key완료-----
.-----26132862 key완료-----
.-----26128552 key완료-----
.-----26141008 key완료-----
.-----26127345 key완료-----
.-----26127097 key완료-----
.-----26162619 key완료-----
.-----26141379 key완료-----
.-----26152372 key완료-----
.-----26157874 key완료-----
.-----26129380 key완료-----
.-----26151344 key완료-----
.-----26149968 key완료-----
.-----26156749 key완료-----
.-----261458

.-----26136304 key완료-----
.-----26141989 key완료-----
.-----26126548 key완료-----
.-----26133674 key완료-----
.-----26135955 key완료-----
.-----26124979 key완료-----
.-----26156104 key완료-----
.-----26144716 key완료-----
.-----26149911 key완료-----
.-----26129287 key완료-----
.-----26124427 key완료-----
.-----26129411 key완료-----
.-----26130534 key완료-----
.-----26140110 key완료-----
.-----26137983 key완료-----
.-----26158194 key완료-----
.-----26134479 key완료-----
.-----26136523 key완료-----
.-----26158831 key완료-----
.-----10210702 key완료-----
.-----26160326 key완료-----
.-----26154458 key완료-----
.-----26154459 key완료-----
.-----26154455 key완료-----
.-----26154456 key완료-----
.-----26154454 key완료-----
.-----26154460 key완료-----
.-----26138857 key완료-----
.-----26158242 key완료-----
.-----26155208 key완료-----
.-----10212343 key완료-----
.-----26148024 key완료-----
.-----26143351 key완료-----
.-----26144553 key완료-----
.-----26157986 key완료-----
.-----26144412 key완료-----
.-----26146430 key완료-----
.-----26146432 key완료-----
.-----102116